# Coleta das bases usadas na análise

Nome: Enzo Cardoso de Almeida Santos Neto

Nome: Luis Costa

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np

## Retirando os tweets

In [ ]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

In [ ]:
#Produto escolhido:
produto = 'Iphone XR'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

## Transformar o excel em um arquivo .txt

In [3]:
base_iphone = pd.read_excel("base_iphoneXR.xlsx")

In [5]:
relevante = base_iphone[base_iphone.Classe == 1]
irrelevante = base_iphone[base_iphone.Classe == 0]

In [6]:
series_relevante = pd.Series(list(relevante.Treinamento))
series_irrelevante = pd.Series(list(irrelevante.Treinamento))

In [7]:
relevante_lista = []
for tweets in series_relevante:
    relevante_lista.append(tweets)

relevante_texto=', '.join(relevante_lista)

with open('relevante.txt','w', encoding="utf8") as arquivo:
    arquivo.write(relevante_texto)

In [8]:
irrelevante_lista = []
for tweets in series_irrelevante:
    irrelevante_lista.append(tweets)
    
irrelevante_texto=', '.join(irrelevante_lista)

with open('irrelevante.txt','w', encoding="utf8") as arquivo:
    arquivo.write(irrelevante_texto)